<a href="https://colab.research.google.com/github/Taeseong-eom/ApplePython/blob/main/dog_vs_cat_ipyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
os. environ['KAGGLE_CONFIG_DIR']= '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

dogs-vs-cats-redux-kernels-edition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [8]:
!unzip -q train.zip -d .

In [12]:
import os
import tensorflow as tf
import shutil

print (len(os.listdir('/content/train/'))) # 학습 이미지 개수가 2만5000개

# 현재 train 폴더에는 개,고양이 사진이 같이 들어있다.
# 개와 고양이 사진을 다른 폴더로 분류해보자.
for i in os.listdir('/content/train/'):
  if 'cat' in i: # 파일 이름에 'cat'이 들어있으면 '/content/dataset/cat/' + i 으로 복사해
    shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
  elif 'dog' in i:# 파일 이름에 'dog'이 들어있으면 '/content/dataset/dog/' + i 으로 복사해
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)

25000


In [23]:

# 이미지를 숫자로 변환해야하는데 opencv라이브러리를 사용하거나
# keras를 사용하면 변환할 수 있다.
# keras가 간편하니 keras를 이용하겠다.
train_ds = tf.keras.preprocessing.image_dataset_from_directory( # 아미지를 데이터셋 자료로 변환
    '/content/dataset/', # 여기에 데이터들이 들어있어
    image_size=(64,64), # 이미지 크기 조정
    batch_size=64, # 이미지를 한번에 2만5천장을 넣지 않고 64장씩 넣겠다.
    subset='training',# 데이터셋에서 20%를 valtidation으로 할당하겠다. 이름은 'training'으로 하겠다.
    validation_split=0.2,
    seed=1234,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory( # 아미지를 데이터셋 자료로 변환
    '/content/dataset/', # 여기에 데이터들이 들어있어
    image_size=(64,64), # 이미지 크기 조정
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234,
)
print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast(i/255.0, tf.float32)
  return i, 정답

#숫자가 0 ~ 255 인걸 0 ~ 1로 압축을 하면 더 빨라진다.
train_ds = train_ds.map(전처리함수) # train_ds의 데이터를 함수에 적용시켜라.
val_ds = val_ds.map(전처리함수)

# import matplotlib.pyplot as plt
# # 데이터셋이 잘 변환되었는지 확인해보자
# for i,정답 in train_ds.take(1): # 64개 데이터가 있는 데이터셋을 하나 가져옴.
#   print(i)
#   print(정답)
#   plt.imshow(i[0].numpy().astype('uint8'))
#   plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu',input_shape=(64,64,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Dropout(0.2), # Overfitting을 완화하기 위해 노트의 20%를 제거.
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'), # convolution을 많이 하면 정확도가 좋아
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')# 0~1로 값 출력
])
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_ds,validation_data=val_ds,epochs=5) # 전처리를 안하면 학습속도, 정확도가 안좋다.

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)      